In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import scipy.stats as stat

print("pandas version : ", pd.__version__)
print("xarray version : ", xr.__version__)

sns.set_style("whitegrid")
plt.rcParams["font.family"] = 'sans-serif'

('pandas version : ', u'0.23.4')
('xarray version : ', u'0.11.0')


### Objective function analysis
VIC parameters were calibrated with NSE, scaled KGEs and annual peak Q error, at each of 490 HCDN basins independently. SCE was used with 20,000 maximum trials.

note:
vic streamflow is mm/day
mhm streamflow is m3/sec

In [2]:
# Getting Data: 
# 1. simulated streamflow from vic and mhm
# 2. runoff output from vic and mhm
# 3. Basin list file

data_path = './data'

# List of objective functions used
obj_fun_list = ['nse','kge','kge_2alpha','kge_5alpha','sig1']
# List of models used
model_list = ['vic', 'mhm']

# Read model simulated streamflow at daily steps
ds_vic_flow_all = xr.open_dataset('%s/results_hcdn_flow_vic_491.nc'%data_path)
ds_mhm_flow_all = xr.open_dataset('%s/results_hcdn_flow_mhm_491.nc'%data_path)

# Read model runoff output i.e., baseflow and surface flow at daily steps
ds_vic_ro_all = xr.open_dataset('%s/results_hcdn_vic_output_491.nc'%data_path)
ds_mhm_ro_all = xr.open_dataset('%s/results_hcdn_mhm_output_491.nc'%data_path)

# Read basin attributes 
df_hcdn = pd.read_csv('%s/hcdn.calib.conus.491.list'%data_path, delim_whitespace=True,
                   header=None, names=['id', 'lat', 'lon', 'area'])    #area is sq-meters

nbasin = len(df_hcdn['id'])
ndata = len(obj_fun_list)

# Print data variables
print('VIC flow data')
print('-------------')
print(ds_vic_flow_all)
print('\nmHM flow data')
print('-------------')
print(ds_mhm_flow_all)
print('\nvic runoff data')
print('-------------')
print(ds_vic_ro_all)
print('\nmHM runoff data')
print('-------------')
print(ds_mhm_ro_all)

VIC flow data
-------------
<xarray.Dataset>
Dimensions:      (hcdn: 491, time: 6940)
Coordinates:
  * hcdn         (hcdn) int64 1022500 1031500 1047000 ... 14362250 14400000
  * time         (time) datetime64[ns] 1989-10-01 1989-10-02 ... 2008-09-30
Data variables:
    nse          (hcdn, time) float32 ...
    kge          (hcdn, time) float32 ...
    kge_alpha    (hcdn, time) float32 ...
    kge_2alpha   (hcdn, time) float32 ...
    kge_5alpha   (hcdn, time) float32 ...
    kge_10alpha  (hcdn, time) float32 ...
    sig1         (hcdn, time) float32 ...
    obs          (hcdn, time) float32 ...

mHM flow data
-------------
<xarray.Dataset>
Dimensions:     (hcdn: 491, time: 6940)
Coordinates:
  * hcdn        (hcdn) int64 1022500 1031500 1047000 ... 14362250 14400000
  * time        (time) datetime64[ns] 1989-10-01 1989-10-02 ... 2008-09-30
Data variables:
    nse         (hcdn, time) float32 ...
    kge_2alpha  (hcdn, time) float32 ...
    kge         (hcdn, time) float32 ...
    sig1 

In [3]:
# Personal functions used here.

def nse(qsim, qobs):
    return 1-np.sum((qsim-qobs)**2)/np.sum((qobs-np.mean(qobs))**2)

    
def corr(qsim, qobs):
    return np.corrcoef(qsim, qobs)[0, 1]
    
    
def alpha(qsim, qobs):
    return math.sqrt(np.sum((qsim-np.mean(qsim))**2)/len(qsim))/math.sqrt(np.sum((qobs-np.mean(qobs))**2)/len(qobs))
    

def beta(qsim, qobs):
    return np.mean(qsim)/np.mean(qobs)


def kge(qsim, qobs):
    return 1-math.sqrt((1-corr(qsim, qobs))**2 + (alpha(qsim, qobs)-1)**2 + (beta(qsim, qobs)-1)**2)


def pbias(qsim, qobs):
    return np.sum((qsim-qobs))/np.sum(qobs)


def bfi(qin, a=0.95):   #baseflow index
    qbs = np.zeros(len(qin))
    for t in np.arange(1,len(qin)):
        qbs[t] = a*qbs[t-1] + (1-a)*(qin[t] + qin[t-1])/2
        if qbs[t] >= qin[t]:
            qbs[t] = qin[t]
    return np.sum(qbs)/np.sum(qin)

In [4]:
# Calibration period
ds_vic_flow_cal = ds_vic_flow_all.sel(time=slice('1999-10-01', '2008-09-30'))
ds_mhm_flow_cal = ds_mhm_flow_all.sel(time=slice('1999-10-01', '2008-09-30'))

#Initialize dictionary
nse_cal = {}
alpha_cal = {}
beta_cal = {} 
corr_cal = {} 
kge_cal = {} 
pbiasFHV_cal = {}
pbiasQpeak_cal = {} 

prob_cal=np.arange(1,float(len(ds_vic_flow_cal['time'])+1))/(1+len(ds_vic_flow_cal['time'])) #probability
for d in range(len(prob_cal)):
    idx50=d
    if prob_cal[d] > 0.5: break
for d in range(len(prob_cal)):
    idx30=d
    if prob_cal[d] > 0.3: break
for d in range(len(prob_cal)):
    idx80=d
    if prob_cal[d] > 0.8: break

for model in ['vic','mhm']:
    nse_array = np.zeros((nbasin, ndata))
    alpha_array = np.zeros((nbasin, ndata))
    beta_array = np.zeros((nbasin, ndata))
    corr_array = np.zeros((nbasin, ndata))
    kge_array = np.zeros((nbasin, ndata))
    pbiasFHV_array = np.zeros((nbasin, ndata))
    pbiasQpeak_array = np.zeros((nbasin, ndata))     
    r=0
    for hid in df_hcdn['id']:
        for c, obj in enumerate(obj_fun_list): 
            if model == 'vic':
                sr_sim = ds_vic_flow_cal[obj].sel(hcdn=hid).to_pandas() 
            elif model =='mhm':
                sr_sim = ds_mhm_flow_cal[obj].sel(hcdn=hid).to_pandas()
                
            sr_sim.where(sr_sim>0.0, 1.0e-7, inplace=True)
            sr_sim_sort = sr_sim.sort_values()
            sr_sim_ann=sr_sim.resample('A-SEP').max()
            if c == 0:
                if model == 'vic':
                    sr_obs = ds_vic_flow_cal['obs'].sel(hcdn=hid).to_pandas()
                elif model =='mhm':
                    sr_obs = ds_mhm_flow_cal['obs'].sel(hcdn=hid).to_pandas()
                sr_obs.where(sr_obs>0.0, 1.0e-7, inplace=True)
                sr_obs_sort = sr_obs.sort_values()
                sr_obs_ann=sr_obs.resample('A-SEP').max()
            
            nse_array[r,c] = nse(sr_sim, sr_obs)
            alpha_array[r,c] = alpha(sr_sim, sr_obs)
            beta_array[r,c] = beta(sr_sim, sr_obs)
            corr_array[r,c] = corr(sr_sim, sr_obs)
            kge_array[r,c] = kge(sr_sim, sr_obs)
            pbiasQpeak_array[r,c] = pbias(sr_sim_ann, sr_obs_ann)
            pbiasFHV_array[r,c] = pbias(sr_sim_sort[idx80:], sr_obs_sort[idx80:])
               
        r += 1
        
    nse_cal[model] = nse_array
    alpha_cal[model] = alpha_array
    beta_cal[model] = beta_array
    corr_cal[model] = corr_array
    kge_cal[model] = kge_array
    pbiasQpeak_cal[model] = pbiasQpeak_array
    pbiasFHV_cal[model] = pbiasFHV_array

In [5]:
# Validation period
ds_vic_flow_val = ds_vic_flow_all.sel(time=slice('1989-10-01', '1999-09-30'))
ds_mhm_flow_val = ds_mhm_flow_all.sel(time=slice('1989-10-01', '1999-09-30'))

#Initialize dictionary
nse_val = {}
alpha_val = {}
beta_val = {} 
corr_val = {} 
kge_val = {} 
pbiasFHV_val = {}
pbiasQpeak_val = {} 

prob_val=np.arange(1,float(len(ds_vic_flow_val['time'])+1))/(1+len(ds_vic_flow_val['time'])) #probability
for d in range(len(prob_val)):
    idx50=d
    if prob_val[d] > 0.5: break
for d in range(len(prob_val)):
    idx30=d
    if prob_val[d] > 0.3: break
for d in range(len(prob_val)):
    idx80=d
    if prob_val[d] > 0.8: break

for model in ['vic','mhm']:
    nse_array = np.zeros((nbasin, ndata))
    alpha_array = np.zeros((nbasin, ndata))
    beta_array = np.zeros((nbasin, ndata))
    corr_array = np.zeros((nbasin, ndata))
    kge_array = np.zeros((nbasin, ndata))
    pbiasFHV_array = np.zeros((nbasin, ndata))
    pbiasQpeak_array = np.zeros((nbasin, ndata))     
    r=0
    for hid in df_hcdn['id']:
        for c, obj in enumerate(obj_fun_list): 
            if model == 'vic':
                sr_sim = ds_vic_flow_val[obj].sel(hcdn=hid).to_pandas() 
            elif model =='mhm':
                sr_sim = ds_mhm_flow_val[obj].sel(hcdn=hid).to_pandas()
                
            sr_sim.where(sr_sim>0.0, 1.0e-7, inplace=True)
            sr_sim_sort = sr_sim.sort_values()
            sr_sim_ann=sr_sim.resample('A-SEP').max()
            if c == 0:
                if model == 'vic':
                    sr_obs = ds_vic_flow_val['obs'].sel(hcdn=hid).to_pandas()
                elif model =='mhm':
                    sr_obs = ds_mhm_flow_val['obs'].sel(hcdn=hid).to_pandas()
                sr_obs.where(sr_obs>0.0, 1.0e-7, inplace=True)
                sr_obs_sort = sr_obs.sort_values()
                sr_obs_ann=sr_obs.resample('A-SEP').max()
            
            nse_array[r,c] = nse(sr_sim, sr_obs)
            alpha_array[r,c] = alpha(sr_sim, sr_obs)
            beta_array[r,c] = beta(sr_sim, sr_obs)
            corr_array[r,c] = corr(sr_sim, sr_obs)
            kge_array[r,c] = kge(sr_sim, sr_obs)
            pbiasQpeak_array[r,c] = pbias(sr_sim_ann, sr_obs_ann)
            pbiasFHV_array[r,c] = pbias(sr_sim_sort[idx80:], sr_obs_sort[idx80:])
               
        r += 1
        
    nse_val[model] = nse_array
    alpha_val[model] = alpha_array
    beta_val[model] = beta_array
    corr_val[model] = corr_array
    kge_val[model] = kge_array
    pbiasQpeak_val[model] = pbiasQpeak_array
    pbiasFHV_val[model] = pbiasFHV_array

In [6]:
# Compute quartiles and median for metrics for each objective function and period
# Multi-level header
iterables = [obj_fun_list, ['cal', 'val']]
index = pd.MultiIndex.from_product(iterables)

#Initialize dictionary
nse_all = {}
alpha_all = {}
beta_all = {} 
corr_all = {} 
kge_all = {} 
pbiasFHV_all = {}
pbiasQpeak_all = {} 

for model in model_list:
    # Combine cal and val arrays
    nse_all_array = np.zeros((nbasin,ndata*2))
    kge_all_array = np.zeros((nbasin,ndata*2))
    alpha_all_array = np.zeros((nbasin,ndata*2))
    beta_all_array = np.zeros((nbasin,ndata*2))
    corr_all_array = np.zeros((nbasin,ndata*2))
    for idx, _ in enumerate(obj_fun_list):
        nse_all_array[:,idx*2] = nse_cal[model][:,idx]
        nse_all_array[:,idx*2+1] = nse_val[model][:,idx]
        kge_all_array[:,idx*2] = kge_cal[model][:,idx]
        kge_all_array[:,idx*2+1] = kge_val[model][:,idx]    
        alpha_all_array[:,idx*2] = alpha_cal[model][:,idx]
        alpha_all_array[:,idx*2+1] = alpha_val[model][:,idx]    
        beta_all_array[:,idx*2] = beta_cal[model][:,idx]
        beta_all_array[:,idx*2+1] = beta_val[model][:,idx]    
        corr_all_array[:,idx*2] = corr_cal[model][:,idx]
        corr_all_array[:,idx*2+1] = corr_val[model][:,idx]
        
    nse_all[model] = nse_all_array
    kge_all[model] = kge_all_array
    alpha_all[model] = alpha_all_array
    beta_all[model] = beta_all_array
    corr_all[model] = corr_all_array
    
# convert numpy arrays to DataFrames
df_nse_vic = pd.DataFrame(nse_all['vic'], index=df_hcdn['id'], columns=index)
df_kge_vic = pd.DataFrame(kge_all['vic'], index=df_hcdn['id'], columns=index)
df_alpha_vic = pd.DataFrame(alpha_all['vic'], index=df_hcdn['id'], columns=index)
df_beta_vic = pd.DataFrame(beta_all['vic'], index=df_hcdn['id'], columns=index)
df_corr_vic = pd.DataFrame(corr_all['vic'], index=df_hcdn['id'], columns=index)

# convert numpy arrays to DataFrames
df_nse_mhm = pd.DataFrame(nse_all['mhm'], index=df_hcdn['id'], columns=index)
df_kge_mhm = pd.DataFrame(kge_all['mhm'], index=df_hcdn['id'], columns=index)
df_alpha_mhm = pd.DataFrame(alpha_all['mhm'], index=df_hcdn['id'], columns=index)
df_beta_mhm = pd.DataFrame(beta_all['mhm'], index=df_hcdn['id'], columns=index)
df_corr_mhm = pd.DataFrame(corr_all['mhm'], index=df_hcdn['id'], columns=index)

# Print quartiles
pd.options.display.float_format = '{:,.2f}'.format
print('VIC')
print('NSE')
print(df_nse_vic.quantile([.25, .5, .75]))
print('\nKGE')
print(df_kge_vic.quantile([.25, .5, .75]))
print('\nalpha')
print(df_alpha_vic.quantile([.25, .5, .75]))
print('\nbeta')
print(df_beta_vic.quantile([.25, .5, .75]))
print('\ncorr')
print(df_corr_vic.quantile([.25, .5, .75]))
print('-------------------')
print('\nmHM')
print('NSE')
print(df_nse_mhm.quantile([.25, .5, .75]))
print('\nKGE')
print(df_kge_mhm.quantile([.25, .5, .75]))
print('\nalpha')
print(df_alpha_mhm.quantile([.25, .5, .75]))
print('\nbeta')
print(df_beta_mhm.quantile([.25, .5, .75]))
print('\ncorr')
print(df_corr_mhm.quantile([.25, .5, .75]))
print('-------------------')

VIC
NSE
      nse       kge      kge_2alpha      kge_5alpha       sig1      
      cal  val  cal  val        cal  val        cal  val   cal   val
0.25 0.49 0.46 0.42 0.41       0.32 0.33       0.26 0.28 -0.54 -0.14
0.50 0.59 0.55 0.52 0.52       0.46 0.48       0.41 0.43 -0.02  0.20
0.75 0.69 0.64 0.64 0.62       0.62 0.60       0.60 0.59  0.29  0.38

KGE
      nse       kge      kge_2alpha      kge_5alpha       sig1      
      cal  val  cal  val        cal  val        cal  val   cal   val
0.25 0.50 0.45 0.53 0.49       0.48 0.48       0.40 0.44 -0.29 -0.00
0.50 0.63 0.59 0.69 0.64       0.67 0.63       0.66 0.61  0.21  0.34
0.75 0.73 0.69 0.80 0.74       0.79 0.74       0.78 0.73  0.52  0.57

alpha
      nse       kge      kge_2alpha      kge_5alpha      sig1     
      cal  val  cal  val        cal  val        cal  val  cal  val
0.25 0.65 0.60 0.72 0.66       0.90 0.76       0.98 0.82 1.13 0.99
0.50 0.74 0.72 0.92 0.83       0.98 0.89       0.99 0.92 1.29 1.15
0.75 0.84 0.82 0.99 0.

In [12]:
df_nse_mhm['kge'].to_csv('mhm.nse.csv')